In [1]:
import pandas as pd
import janitor

In [2]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

In [3]:
data = pd.read_csv('filter.csv')

In [4]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [5]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

### Note
~~Daily report for april 20 (ie, reported in 21 april) stops using testing site,~~
<br>
~~So we shall treat both sets of data accordingly~~

In [28]:
tbl1 = data.filter_date('FECHA_INGRESO','2020-01-06','2020-04-25')
tbl1 = tbl1.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl1 = tbl1.set_index([
    'FECHA_INGRESO',
    'FECHA_SINTOMAS',
    'FECHA_DEF',
    'ENTIDAD_UM',
    'ID_REGISTRO',
])

In [29]:
# Health testing site (positive) cases by date
tbl1 = tbl1.groupby([
    'FECHA_INGRESO',
    'FECHA_SINTOMAS',
    'FECHA_DEF',
    'ENTIDAD_UM',
    'ID_REGISTRO',
], as_index=True).count().reset_index()

In [30]:
tbl1

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,ENTIDAD_UM,ID_REGISTRO,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,6
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,5
2,2020-01-27,2020-01-27,9999-99-99,15,1879e0,1
3,2020-02-07,2020-02-07,9999-99-99,11,03e780,3
4,2020-02-12,2020-02-07,9999-99-99,2,07b5c5,6
...,...,...,...,...,...,...
14745,2020-04-24,2020-04-23,9999-99-99,5,140deb,1
14746,2020-04-24,2020-04-23,9999-99-99,19,0e0ca6,1
14747,2020-04-24,2020-04-23,9999-99-99,27,08a84b,1
14748,2020-04-24,2020-04-23,9999-99-99,29,1a0b7e,1


In [31]:
tbl2

,FECHA_INGRESO,Region_ID,ID_REGISTRO,FECHA_DEF,RESULTADO
0,2020-04-21,1,02de72,2,2
1,2020-04-21,1,02f180,2,2
2,2020-04-21,1,0c19cb,2,2
3,2020-04-21,1,0d5997,2,2
4,2020-04-21,1,0db7a7,2,2
...,...,...,...,...,...
1587,2020-04-24,27,1add7e,1,1
1588,2020-04-24,29,06a593,1,1
1589,2020-04-24,29,06b382,1,1
1590,2020-04-24,29,09d6b7,1,1


In [32]:
tbl2 = data.filter_date('FECHA_INGRESO','2020-04-21','2020-04-25')
tbl2 = tbl2.drop(['ENTIDAD_RES'], axis=1) # drop by medical unit
tbl2 = tbl2.set_index([
    'FECHA_INGRESO',
    'FECHA_SINTOMAS',
    'ENTIDAD_UM',
    'ID_REGISTRO',
])

In [33]:
# Residence cases (positive) by date
tbl2 = tbl2.groupby(['FECHA_INGRESO','ENTIDAD_UM','ID_REGISTRO'], as_index=True).count().reset_index()

In [34]:
# Join tables

In [35]:
tbl1 = tbl1.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [36]:
tbl2 = tbl2.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [37]:
tbl_join = pd.concat([tbl1, tbl2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
# Friendly colnames

In [39]:
data.keys()

Index(['ID_REGISTRO', 'ENTIDAD_UM', 'ENTIDAD_RES', 'FECHA_INGRESO',
       'FECHA_SINTOMAS', 'FECHA_DEF', 'RESULTADO'],
      dtype='object')

In [40]:
tbl_join = tbl_join.rename(columns={'FECHA_INGRESO':'Date_Confirmed', 'ID_REGISTRO':'Case_ID', 'RESULTADO':'Status'})

In [41]:
len(tbl_join) # Total cases would have been less if we didn't take into account the change in test reporting :o

16342

In [42]:
tbl_join['Status'] = '1'

In [ ]:
# ^^^ fix above

In [43]:
tbl1.to_csv('confirmed_by_date.csv', index=False)

In [44]:
tbl1

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,6
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,5
2,2020-01-27,2020-01-27,9999-99-99,15,1879e0,1
3,2020-02-07,2020-02-07,9999-99-99,11,03e780,3
4,2020-02-12,2020-02-07,9999-99-99,2,07b5c5,6
...,...,...,...,...,...,...
14745,2020-04-24,2020-04-23,9999-99-99,5,140deb,1
14746,2020-04-24,2020-04-23,9999-99-99,19,0e0ca6,1
14747,2020-04-24,2020-04-23,9999-99-99,27,08a84b,1
14748,2020-04-24,2020-04-23,9999-99-99,29,1a0b7e,1
